In [37]:
#Import Dependencies
from pylab import *
import os
import pandas as pd
# import librosa
# import librosa.display
# import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Model

In [38]:
df = pd.read_csv('../data/voice_data.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,label,emotions
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280939,-8.379515,-0.323866,...,-1.091130,-2.829661,-1.520689,-2.216941,-3.051777,1.104535,0.969608,2.822009,6,fearful
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,-0.281293,0.684120,0.073632,2.641415,0.838373,1.800676,-0.601248,1.839665,5,angry
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,-1.439343,0.234170,0.237684,0.759627,0.519322,0.865444,1.468920,1.691772,8,surprise
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,-0.685990,0.215328,-0.507966,0.684138,0.205525,0.921709,0.126503,0.312205,6,fearful
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193613,-8.719680,-7.731156,-8.692744,0.833810,...,0.724145,1.723078,1.540662,1.585126,0.513308,0.538170,-0.266717,-1.757481,5,angry


In [40]:
df.drop(inplace=True, columns=["emotions"])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280939,-8.379515,-0.323866,...,-0.138160,-1.091130,-2.829661,-1.520689,-2.216941,-3.051777,1.104535,0.969608,2.822009,6
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,3.469327,-0.281293,0.684120,0.073632,2.641415,0.838373,1.800676,-0.601248,1.839665,5
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,0.658183,-1.439343,0.234170,0.237684,0.759627,0.519322,0.865444,1.468920,1.691772,8
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,0.015738,-0.685990,0.215328,-0.507966,0.684138,0.205525,0.921709,0.126503,0.312205,6
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193613,-8.719680,-7.731156,-8.692744,0.833810,...,2.204446,0.724145,1.723078,1.540662,1.585126,0.513308,0.538170,-0.266717,-1.757481,5


In [42]:
target = df["label"]

In [43]:
data = df.drop("label", axis=1)
feature_names = data.columns
data.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280939,-8.379515,-0.323866,...,-4.505692,-0.138160,-1.091130,-2.829661,-1.520689,-2.216941,-3.051777,1.104535,0.969608,2.822009
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,3.918904,3.469327,-0.281293,0.684120,0.073632,2.641415,0.838373,1.800676,-0.601248,1.839665
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,-1.102092,0.658183,-1.439343,0.234170,0.237684,0.759627,0.519322,0.865444,1.468920,1.691772
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,0.062081,0.015738,-0.685990,0.215328,-0.507966,0.684138,0.205525,0.921709,0.126503,0.312205
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193613,-8.719680,-7.731156,-8.692744,0.833810,...,0.995639,2.204446,0.724145,1.723078,1.540662,1.585126,0.513308,0.538170,-0.266717,-1.757481


In [44]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [78]:
y_train

1413    7
1556    7
2639    2
1189    8
2299    3
       ..
3444    6
466     6
3092    5
3772    2
860     4
Name: label, Length: 3180, dtype: int64

In [79]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=256, activation='relu', input_dim=2))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [106]:
model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_42 (Dense)             (None, 40)                1640      
Total params: 3,280
Trainable params: 3,280
Non-trainable params: 0
_________________________________________________________________


In [111]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [114]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100


ValueError: in user code:

    /Users/timsamson/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/timsamson/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/timsamson/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/timsamson/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/timsamson/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_e